In [1]:

# imports
import os
import sys
import types
import json

# figure size/format
fig_width = 10
fig_height = 5
fig_format = 'retina'
fig_dpi = 96

# matplotlib defaults / format
try:
  import matplotlib.pyplot as plt
  plt.rcParams['figure.figsize'] = (fig_width, fig_height)
  plt.rcParams['figure.dpi'] = fig_dpi
  plt.rcParams['savefig.dpi'] = fig_dpi
  from IPython.display import set_matplotlib_formats
  set_matplotlib_formats(fig_format)
except Exception:
  pass

# plotly use connected mode
try:
  import plotly.io as pio
  pio.renderers.default = "notebook_connected"
except Exception:
  pass

# enable pandas latex repr when targeting pdfs
try:
  import pandas as pd
  if fig_format == 'pdf':
    pd.set_option('display.latex.repr', True)
except Exception:
  pass



# output kernel dependencies
kernel_deps = dict()
for module in list(sys.modules.values()):
  # Some modules play games with sys.modules (e.g. email/__init__.py
  # in the standard library), and occasionally this can cause strange
  # failures in getattr.  Just ignore anything that's not an ordinary
  # module.
  if not isinstance(module, types.ModuleType):
    continue
  path = getattr(module, "__file__", None)
  if not path:
    continue
  if path.endswith(".pyc") or path.endswith(".pyo"):
    path = path[:-1]
  if not os.path.exists(path):
    continue
  kernel_deps[path] = os.stat(path).st_mtime
print(json.dumps(kernel_deps))

# set run_path if requested
if r'':
  os.chdir(r'')

# reset state
%reset

def ojs_define(**kwargs):
  import json
  from IPython.core.display import display, HTML

  # do some minor magic for convenience when handling pandas
  # dataframes
  def convert(v):
    try:
      import pandas as pd
    except ModuleNotFoundError: # don't do the magic when pandas is not available
      return v
    if type(v) == pd.DataFrame:
      j = json.loads(v.T.to_json(orient='split'))
      return dict((k,v) for (k,v) in zip(j["index"], j["data"]))
    else:
      return v
  
  v = dict(contents=list(dict(name=key, value=convert(value)) for (key, value) in kwargs.items()))
  display(HTML('<script type="ojs-define">' + json.dumps(v) + '</script>'), metadata=dict(ojs_define = True))
globals()["ojs_define"] = ojs_define


C:\Users\kerry\AppData\Local\Temp\ipykernel_16548\3609341226.py:20: DeprecationWarning: `set_matplotlib_formats` is deprecated since IPython 7.23, directly use `matplotlib_inline.backend_inline.set_matplotlib_formats()`
  set_matplotlib_formats(fig_format)


{"C:\\Users\\kerry\\AppData\\Local\\Programs\\Python\\Python310\\lib\\importlib\\_bootstrap.py": 1647454514.0, "C:\\Users\\kerry\\AppData\\Local\\Programs\\Python\\Python310\\lib\\importlib\\_bootstrap_external.py": 1647454514.0, "C:\\Users\\kerry\\AppData\\Local\\Programs\\Python\\Python310\\lib\\codecs.py": 1647454514.0, "C:\\Users\\kerry\\AppData\\Local\\Programs\\Python\\Python310\\lib\\encodings\\aliases.py": 1647454514.0, "C:\\Users\\kerry\\AppData\\Local\\Programs\\Python\\Python310\\lib\\encodings\\__init__.py": 1647454514.0, "C:\\Users\\kerry\\AppData\\Local\\Programs\\Python\\Python310\\lib\\encodings\\utf_8.py": 1647454514.0, "C:\\Users\\kerry\\AppData\\Local\\Programs\\Python\\Python310\\lib\\encodings\\cp1252.py": 1647454514.0, "C:\\Users\\kerry\\AppData\\Local\\Programs\\Python\\Python310\\lib\\abc.py": 1647454514.0, "C:\\Users\\kerry\\AppData\\Local\\Programs\\Python\\Python310\\lib\\io.py": 1647454514.0, "C:\\Users\\kerry\\AppData\\Local\\Programs\\Python\\Python310\\li

In [2]:
import pandas as pd
import pymssql
from sqlalchemy import create_engine

server = 'fs.rice.edu'
database = 'stocks'
username = 'stocks'
password = '6LAZH1'

string = "mssql+pymssql://" + username + ":" + password + "@" + server + "/" + database 
conn = create_engine(string).connect()

df = pd.read_sql(
    "select * from information_schema.tables",
    conn
)
df

,TABLE_CATALOG,TABLE_SCHEMA,TABLE_NAME,TABLE_TYPE
0,stocks,dbo,sep2,BASE TABLE
1,stocks,dbo,today,BASE TABLE
2,stocks,dbo,indicators,BASE TABLE
3,stocks,dbo,tickers,BASE TABLE
4,stocks,dbo,sf1,BASE TABLE
5,stocks,dbo,sep,BASE TABLE
6,stocks,dbo,daily,BASE TABLE


In [3]:
df = pd.read_sql(
    "select * from indicators",
    conn
)
df.to_excel("indicators.xlsx")

In [4]:
df = pd.read_sql(
    """
    select * from sf1 
    where ticker='AAPL' and dimension='ARY'
    order by reportperiod
    """, 
    conn
)
df

,ticker,dimension,calendardate,datekey,reportperiod,lastupdated,accoci,assets,assetsavg,assetsc,...,sharesbas,shareswa,shareswadil,sps,tangibles,taxassets,taxexp,taxliabilities,tbvps,workingcapital
0,AAPL,ARY,2019-12-31,2019-10-31,2019-09-28,2022-10-28,-5.840000e+08,3.385160e+11,3.441180e+11,1.628190e+11,...,1.777306e+10,1.847134e+10,1.859565e+10,14.085,3.385160e+11,0.0,1.048100e+10,0.0,18.327,5.710100e+10
1,AAPL,ARY,2020-12-31,2020-10-30,2020-09-26,2022-10-28,-4.060000e+08,3.238880e+11,3.255625e+11,1.437130e+11,...,1.700180e+10,1.735212e+10,1.752821e+10,15.820,3.238880e+11,0.0,9.680000e+09,0.0,18.666,3.832100e+10
2,AAPL,ARY,2021-12-31,2021-10-29,2021-09-25,2022-10-28,1.630000e+08,3.510020e+11,3.430135e+11,1.348360e+11,...,1.640640e+10,1.670127e+10,1.686492e+10,21.904,3.510020e+11,0.0,1.452700e+10,0.0,21.016,9.355000e+09
3,AAPL,ARY,2022-12-31,2022-10-28,2022-09-24,2022-10-28,-1.110900e+10,3.527550e+11,3.552292e+11,1.354050e+11,...,1.590812e+10,1.621596e+10,1.632582e+10,24.317,3.527550e+11,0.0,1.930000e+10,0.0,21.754,-1.857700e+10


In [5]:
df = pd.read_sql(
    """
    select * from sf1 
    where ticker in ('AAPL', 'CVX') and dimension='ARQ' and reportperiod>='2020-01-01'
    order by ticker, reportperiod
    """, 
    conn
)
df

,ticker,dimension,calendardate,datekey,reportperiod,lastupdated,accoci,assets,assetsavg,assetsc,...,sharesbas,shareswa,shareswadil,sps,tangibles,taxassets,taxexp,taxliabilities,tbvps,workingcapital
0,AAPL,ARQ,2020-03-31,2020-05-01,2020-03-28,2022-10-28,-2.789000e+09,3.204000e+11,None,1.437530e+11,...,1.733734e+10,1.744040e+10,1.761876e+10,3.344,3.204000e+11,0.0,1.886000e+09,0.000000e+00,18.371,4.765900e+10
1,AAPL,ARQ,2020-06-30,2020-07-31,2020-06-27,2022-10-28,-5.500000e+08,3.173440e+11,None,1.400650e+11,...,1.710254e+10,1.725029e+10,1.741915e+10,3.460,3.173440e+11,0.0,1.884000e+09,0.000000e+00,18.396,4.474700e+10
2,AAPL,ARQ,2020-09-30,2020-10-30,2020-09-26,2022-10-28,-4.060000e+08,3.238880e+11,None,1.437130e+11,...,1.700180e+10,1.705762e+10,1.725652e+10,3.793,3.238880e+11,0.0,2.228000e+09,0.000000e+00,18.988,3.832100e+10
3,AAPL,ARQ,2020-12-31,2021-01-28,2020-12-26,2022-10-28,1.790000e+08,3.540540e+11,None,1.541060e+11,...,1.678810e+10,1.693512e+10,1.711369e+10,6.580,3.540540e+11,0.0,4.824000e+09,0.000000e+00,20.906,2.159900e+10
4,AAPL,ARQ,2021-03-31,2021-04-29,2021-03-27,2022-10-28,-2.860000e+08,3.371580e+11,None,1.214650e+11,...,1.668763e+10,1.675348e+10,1.692916e+10,5.347,3.371580e+11,0.0,4.381000e+09,0.000000e+00,20.125,1.508000e+10
5,AAPL,ARQ,2021-06-30,2021-07-28,2021-06-26,2022-10-28,5.800000e+07,3.298400e+11,None,1.144230e+11,...,1.653017e+10,1.662937e+10,1.678174e+10,4.897,3.298400e+11,0.0,2.625000e+09,0.000000e+00,19.835,6.669000e+09
6,AAPL,ARQ,2021-09-30,2021-10-29,2021-09-25,2022-10-28,1.630000e+08,3.510020e+11,None,1.348360e+11,...,1.640640e+10,1.648712e+10,1.663510e+10,5.056,3.510020e+11,0.0,2.697000e+09,0.000000e+00,21.289,9.355000e+09
7,AAPL,ARQ,2021-12-31,2022-01-28,2021-12-25,2022-10-28,-9.270000e+08,3.811910e+11,None,1.531540e+11,...,1.631944e+10,1.639172e+10,1.651929e+10,7.561,3.811910e+11,0.0,6.611000e+09,0.000000e+00,23.255,5.580000e+09
8,AAPL,ARQ,2022-03-31,2022-04-29,2022-03-26,2022-10-28,-6.494000e+09,3.506620e+11,None,1.181800e+11,...,1.618518e+10,1.627880e+10,1.640332e+10,5.976,3.506620e+11,0.0,5.129000e+09,0.000000e+00,21.541,-9.328000e+09
9,AAPL,ARQ,2022-06-30,2022-07-29,2022-06-25,2022-10-28,-9.297000e+09,3.363090e+11,None,1.122920e+11,...,1.607075e+10,1.616294e+10,1.626220e+10,5.133,3.363090e+11,0.0,3.624000e+09,0.000000e+00,20.807,-1.758100e+10


In [6]:
df = pd.read_sql(
    """
    select * from sep where ticker='AAPL' 
    order by date
    """,
    conn
)
df

,ticker,date,lastupdated,open_,high,low,close_,volume,closeadj,closeunadj
0,AAPL,1997-12-31,2022-12-27,0.117,0.122,0.116,0.117,4.063584e+08,0.100,13.13
1,AAPL,1998-01-02,2022-12-27,0.122,0.145,0.120,0.145,7.181104e+08,0.123,16.25
2,AAPL,1998-01-05,2022-12-27,0.147,0.148,0.136,0.142,6.518736e+08,0.121,15.88
3,AAPL,1998-01-06,2022-12-27,0.142,0.178,0.132,0.169,1.812474e+09,0.144,18.94
4,AAPL,1998-01-07,2022-12-27,0.168,0.170,0.154,0.156,1.041622e+09,0.133,17.50
...,...,...,...,...,...,...,...,...,...,...
6296,AAPL,2023-01-09,2023-01-09,130.465,133.410,129.890,130.150,7.049660e+07,130.150,130.15
6297,AAPL,2023-01-10,2023-01-10,130.260,131.264,128.120,130.730,6.342765e+07,130.730,130.73
6298,AAPL,2023-01-11,2023-01-11,131.250,133.510,130.460,133.490,6.911440e+07,133.490,133.49
6299,AAPL,2023-01-12,2023-01-12,133.880,134.260,131.440,133.410,7.124770e+07,133.410,133.41


In [7]:
df = pd.read_sql(
    """
    select * from daily where ticker='AAPL' 
    order by date
    """,
    conn
)
df

,ticker,date,lastupdated,ev,evebit,evebitda,marketcap,pb,pe,ps
0,AAPL,1998-12-01,2020-08-31,4349.3,-4.2,-4.7,4599.3,3.1,-4.4,0.6
1,AAPL,1998-12-02,2020-08-31,4605.6,-4.4,-5.0,4855.6,3.3,-4.6,0.7
2,AAPL,1998-12-03,2020-08-31,4289.0,-4.1,-4.6,4539.0,3.1,-4.3,0.6
3,AAPL,1998-12-04,2020-08-31,4153.2,-4.0,-4.5,4403.2,3.0,-4.2,0.6
4,AAPL,1998-12-07,2020-08-31,4289.0,-4.1,-4.6,4539.0,3.1,-4.3,0.6
...,...,...,...,...,...,...,...,...,...,...
6065,AAPL,2023-01-09,2023-01-09,2166864.6,17.8,16.3,2070441.6,40.9,20.7,5.3
6066,AAPL,2023-01-10,2023-01-10,2176091.3,17.8,16.3,2079668.3,41.0,20.8,5.3
6067,AAPL,2023-01-11,2023-01-11,2219997.7,18.2,16.7,2123574.7,41.9,21.3,5.4
6068,AAPL,2023-01-12,2023-01-12,2218725.0,18.2,16.7,2122302.0,41.9,21.3,5.4
